In [1]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

from langchain import hub

In [2]:
# Create your prompt template
prompt = """You are a assistant for question answering tasks.
use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Answer in bullet points.make sure to use the context provided.
Question : {question}
Context:{context}
Answer:
"""

prompt = ChatPromptTemplate.from_template(prompt)

In [3]:
llm = ChatOllama(model="qwen3:1.7b",base_url="http://localhost:11434")
llm.invoke("Hi")

AIMessage(content='<think>\nOkay, the user said "Hi". I need to respond politely. Let me think about the best way to reply.\n\nFirst, acknowledge their greeting. Maybe say "Hello!" to keep it friendly. Then offer assistance. They might be testing if I\'m responsive or just starting a conversation. I should keep it open-ended so they feel comfortable sharing more.\n\nI should avoid any markdown and keep the response simple. No need for extra explanations unless they ask. Just a straightforward greeting and offer to help. Make sure the tone is warm and approachable. Alright, that should cover it.\n</think>\n\nHello! How can I assist you today? 😊', additional_kwargs={}, response_metadata={'model': 'qwen3:1.7b', 'created_at': '2025-08-24T03:34:32.4202507Z', 'done': True, 'done_reason': 'stop', 'total_duration': 8088521500, 'load_duration': 4816792800, 'prompt_eval_count': 9, 'prompt_eval_duration': 490568800, 'eval_count': 133, 'eval_duration': 2772004100, 'model_name': 'qwen3:1.7b'}, id='

In [6]:
# Example mock retriever for demonstration purposes
from langchain_core.runnables import RunnableLambda

class MockDoc:
    def __init__(self, page_content):
        self.page_content = page_content

def mock_retriever(input):
    # Returns a list of mock documents
    return [MockDoc("This is a sample context document.")]

retriever = RunnableLambda(mock_retriever)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

format_docs_runnable = RunnableLambda(format_docs)

rag_chain = (
    {"context": retriever | format_docs_runnable, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)